A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente. O EV/EBIT compara o Valor da Empresa (Enterprise Value, EV = Capitalização de mercado + Dívida total - Caixa e equivalentes de caixa) ao seu Lucro Antes de Juros e Impostos (Earnings Before Interest and Taxes, EBIT), o EV leva em conta o valor de mercado total da empresa, adicionando as dívidas e subtraindo o caixa, oferecendo assim, uma visão mais abrangente do valor da empresa do que simplesmente seu valor de mercado. O EBIT é uma medida da capacidade de geração de lucro operacional da empresa, excluindo os efeitos das decisões de estrutura de capital e impostos. Um EV/EBIT menor sugere que o preço da ação da empresa pode estar baixo em comparação com a quantidade de lucro operacional que está gerando, indicando que a empresa está subavaliada pelo mercado. No entanto, é importante analisar o EV/EBIT dentro do contexto de cada setor, já que diferentes indústrias têm padrões de avaliação distintos.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render. O ROIC é um dos indicadores financeiros mais importantes para analisar a eficiência operacional de uma empresa. Para calcular o ROIC, divide-se o lucro operacional líquido de impostos pelo capital investido total, que inclui dívidas e patrimônio líquido. Um ROIC alto é geralmente interpretado como um sinal de que a empresa está utilizando seu capital de maneira eficaz para gerar lucros.

ROIC utilizada por Greenblatt é diferente da fórmula tradicional

ROIC = EBIT / (Capital Circulante Líquido + Ativos Fixos Líquidos)

Onde o Capital circulante Líquido é um proxy do Capital de Giro da Empresa (Ativo Circulante – Passivo Circulante), os Ativos Fixos Líquidos é proxy dos ativos permanentes (imobilizado e intangíveis, utilizados para gerar retorno no longo prazo. A Taxa de Retorno de Lucros para uma empresa é a fórmula tradicional: Taxa de Retorno = Lucro Operacional / Valor da Empresa

Estes dois indicadores são colocados cada um em um ranking, da melhor (rank 1) para a pior empresa. Depois somamos os números das duas colunas, quanto menor a soma, melhor.

Para os princípios de Greenblatt, empresas pertencentes aos setores de utilidades públicas, bancos e  seguradoras devem ser excluídas da seleção. 

In [95]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
from IPython.display import display, HTML


def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))


def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None

def setor(setor:int):

    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    content = requests.get(url, headers=hdr)
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]
    return list(df['Papel'])



dados do site https://www.fundamentus.com.br/resultado.php

In [96]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
data_funds = pd.DataFrame(funda)
data_funds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [97]:
earning_yield = 'EV/EBIT'  #Quanto menor melhor
return_on_capital = 'ROIC' #ROIC – Quanto maior melhor

### filtros e ranking

- Empresa precisa ter ROIC positivo
- Empresa precisa ter Lucro Operacional positivo.
- Volume de negociação diário medio maior que R$ 1000000.000,00 nos últimos 2 meses.

ROIC – Quanto maior melhor

EV/EBIT – Quanto menor melhor

In [98]:
funds = data_funds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]
funds =  funds[funds['Liq.2meses'] > 1000000]     #Volume diário médio negociado nos últimos 2 meses

""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1

### lista ranqueada

In [99]:
funds.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a',
       'Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final'],
      dtype='object')

In [100]:
funds[['Papel','EV/EBITDA', 'ROIC', 'Liq.2meses','Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final']].head(20)

,Papel,EV/EBITDA,ROIC,Liq.2meses,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,0.56,78.18,9.025130e+07,1.0,1.0,2.0
2,WIZC3,1.92,35.46,3.350840e+06,2.0,5.0,7.0
3,LREN3,2.61,22.96,2.854290e+08,7.0,13.0,20.0
4,PLPL3,3.88,29.26,2.063220e+07,11.0,9.0,20.0
5,QUAL3,1.74,19.33,7.326760e+06,3.0,20.0,23.0
6,ASAI3,2.91,19.41,1.379140e+08,9.0,19.0,28.0
7,PETR4,2.53,17.85,1.254740e+09,6.0,23.0,29.0
8,CMIN3,3.81,26.67,4.205230e+07,18.0,11.0,29.0
9,PETR3,2.61,17.85,3.890580e+08,8.0,23.0,31.0
10,CSUD3,3.64,20.52,1.093700e+06,27.0,14.0,41.0


retira banco, seguradora, utilidade publica (agua e luz - [util-b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-utilidade-publica-util-composicao-da-carteira.htm))


In [101]:
bancos = setor(20)
print(*bancos)
seguros = setor(31)
print(*seguros)
util_agua = setor(2)
util_luz = setor(14)
util=util_agua+util_luz
print(*util)


BNBR3 BAZA3 BSLI3 BSLI4 BRSR6 BRSR5 BGIP4 BRSR3 BBAS3 BGIP3 BMGB4 BEES3 ABCB4 BEES4 BBDC3 BMEB3 ITSA4 ITSA3 SANB3 BBDC4 BPAC5 PINE3 SANB11 BMEB4 SANB4 PINE4 ITUB3 ITUB4 BMIN4 BRBI11 BPAC11 MERC4 MERC3 BPAN4 BPAC3 RPAD6 RPAD3 RPAD5
WIZC3 BBSE3 PSSA3 IRBR3 CXSE3
AMBP3 SAPR4 SAPR11 SAPR3 CSMG3 SBSP3 ORVR3 GPAR3 AXIA6 AXIA3 AURE3 RIOS3 RNEW3 RNEW4 CEED3 LIGT3 CEEB5 EMAE4 CEEB3 CBEE3 ISAE4 CLSC3 ENGI4 CLSC4 EQMA3B ENGI11 ISAE3 EQPA3 CMIG4 TAEE3 EKTR4 TAEE11 TAEE4 ALUP4 EQPA6 NEOE3 ALUP11 ENGI3 COCE3 CEBR5 COCE5 CEBR3 EKTR3 ALUP3 CMIG3 CEBR6 CPFE3 REDE3 EGIE3 ENMT3 ENMT4 EQPA5 AFLT3 EQTL3 GEPA3 GEPA4 CPLE3 CPLE5 CPLE6 SRNA3 ENEV3


#### sem bancos

In [102]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(bancos)],10
)
          

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,47.91,9.64,2.08,0.805,3.90,0.581,4.69,0.98,-4.82,0.57,0.56,81.71,8.47,1.26,78.18,21.54,90251300.0,1.491500e+10,0.00,17.08,1.0,1.0,2.0,PSSA
WIZC3,9.09,7.65,2.08,1.064,2.75,0.602,38.35,2.10,-2.65,2.23,1.92,50.71,25.66,1.07,35.46,27.23,3350840.0,6.978480e+08,0.55,13.39,2.0,5.0,7.0,WIZC
LREN3,13.65,9.87,1.35,0.881,6.46,0.749,3.00,3.94,5.70,3.58,2.61,22.36,8.92,1.76,22.96,13.72,285429000.0,1.014310e+10,0.04,10.59,7.0,13.0,20.0,LREN
PLPL3,13.97,9.01,2.88,0.989,7.20,0.744,1.18,3.88,5.06,3.99,3.88,25.48,13.28,3.70,29.26,31.96,20632200.0,9.895800e+08,1.06,25.95,11.0,9.0,20.0,PLPL
QUAL3,2.23,64.91,0.48,0.429,0.25,0.162,-2.03,1.09,-0.53,2.70,1.74,39.27,1.14,0.81,19.33,0.74,7326760.0,1.325600e+09,1.05,-8.82,3.0,20.0,23.0,QUAL
ASAI3,7.26,10.70,1.72,0.128,1.48,0.218,5.97,1.72,-0.42,3.80,2.91,7.44,1.20,1.12,19.41,16.05,137914000.0,5.720000e+09,2.85,29.34,9.0,19.0,28.0,ASAI
PETR4,30.41,5.06,0.93,0.798,10.63,0.323,-11.88,1.97,-0.61,3.55,2.53,40.44,15.87,0.82,17.85,18.33,1254740000.0,4.229340e+11,0.89,2.45,6.0,23.0,29.0,PETR
CMIN3,5.45,12.10,2.96,1.668,8.16,0.823,3.53,5.46,-3.16,4.66,3.81,30.54,13.78,2.02,26.67,24.47,42052300.0,1.009740e+10,0.91,0.24,18.0,11.0,29.0,CMIN
CSUD3,17.15,7.61,1.41,1.185,16.25,0.960,15.67,5.94,368.94,5.52,3.64,19.95,15.58,1.24,20.52,18.49,1093700.0,5.097310e+08,0.10,4.34,27.0,14.0,41.0,CSUD
CEAB3,12.69,7.40,1.11,0.485,7.12,0.439,2.76,3.85,-3.46,3.92,2.91,12.60,6.56,1.50,16.14,14.95,133664000.0,3.536830e+09,0.35,11.98,10.0,32.0,42.0,CEAB


#### sem seguradora

In [103]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(seguros)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
LREN3,13.65,9.87,1.35,0.881,6.46,0.749,3.00,3.94,5.70,3.58,2.61,22.36,8.92,1.76,22.96,13.72,285429000.0,1.014310e+10,0.04,10.59,7.0,13.0,20.0,LREN
PLPL3,13.97,9.01,2.88,0.989,7.20,0.744,1.18,3.88,5.06,3.99,3.88,25.48,13.28,3.70,29.26,31.96,20632200.0,9.895800e+08,1.06,25.95,11.0,9.0,20.0,PLPL
QUAL3,2.23,64.91,0.48,0.429,0.25,0.162,-2.03,1.09,-0.53,2.70,1.74,39.27,1.14,0.81,19.33,0.74,7326760.0,1.325600e+09,1.05,-8.82,3.0,20.0,23.0,QUAL
ASAI3,7.26,10.70,1.72,0.128,1.48,0.218,5.97,1.72,-0.42,3.80,2.91,7.44,1.20,1.12,19.41,16.05,137914000.0,5.720000e+09,2.85,29.34,9.0,19.0,28.0,ASAI
PETR4,30.41,5.06,0.93,0.798,10.63,0.323,-11.88,1.97,-0.61,3.55,2.53,40.44,15.87,0.82,17.85,18.33,1254740000.0,4.229340e+11,0.89,2.45,6.0,23.0,29.0,PETR
CMIN3,5.45,12.10,2.96,1.668,8.16,0.823,3.53,5.46,-3.16,4.66,3.81,30.54,13.78,2.02,26.67,24.47,42052300.0,1.009740e+10,0.91,0.24,18.0,11.0,29.0,CMIN
CSUD3,17.15,7.61,1.41,1.185,16.25,0.960,15.67,5.94,368.94,5.52,3.64,19.95,15.58,1.24,20.52,18.49,1093700.0,5.097310e+08,0.10,4.34,27.0,14.0,41.0,CSUD
CEAB3,12.69,7.40,1.11,0.485,7.12,0.439,2.76,3.85,-3.46,3.92,2.91,12.60,6.56,1.50,16.14,14.95,133664000.0,3.536830e+09,0.35,11.98,10.0,32.0,42.0,CEAB
LEVE3,34.22,7.95,5.29,0.859,7.95,1.177,7.72,5.29,-8.55,6.33,5.54,16.24,10.81,1.31,34.85,66.51,11875700.0,8.769090e+08,1.71,9.91,42.0,6.0,48.0,LEVE
BEEF3,5.72,-7.02,4.43,0.111,2.88,0.126,0.78,1.61,-0.38,4.93,3.91,6.92,-1.57,1.35,17.04,-63.09,100235000.0,1.291830e+09,20.66,14.53,21.0,28.0,49.0,BEEF


#### sem utilidade publica

In [104]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(util)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,47.91,9.64,2.08,0.805,3.90,0.581,4.69,0.98,-4.82,0.57,0.56,81.71,8.47,1.26,78.18,21.54,90251300.0,1.491500e+10,0.00,17.08,1.0,1.0,2.0,PSSA
WIZC3,9.09,7.65,2.08,1.064,2.75,0.602,38.35,2.10,-2.65,2.23,1.92,50.71,25.66,1.07,35.46,27.23,3350840.0,6.978480e+08,0.55,13.39,2.0,5.0,7.0,WIZC
LREN3,13.65,9.87,1.35,0.881,6.46,0.749,3.00,3.94,5.70,3.58,2.61,22.36,8.92,1.76,22.96,13.72,285429000.0,1.014310e+10,0.04,10.59,7.0,13.0,20.0,LREN
PLPL3,13.97,9.01,2.88,0.989,7.20,0.744,1.18,3.88,5.06,3.99,3.88,25.48,13.28,3.70,29.26,31.96,20632200.0,9.895800e+08,1.06,25.95,11.0,9.0,20.0,PLPL
QUAL3,2.23,64.91,0.48,0.429,0.25,0.162,-2.03,1.09,-0.53,2.70,1.74,39.27,1.14,0.81,19.33,0.74,7326760.0,1.325600e+09,1.05,-8.82,3.0,20.0,23.0,QUAL
ASAI3,7.26,10.70,1.72,0.128,1.48,0.218,5.97,1.72,-0.42,3.80,2.91,7.44,1.20,1.12,19.41,16.05,137914000.0,5.720000e+09,2.85,29.34,9.0,19.0,28.0,ASAI
PETR4,30.41,5.06,0.93,0.798,10.63,0.323,-11.88,1.97,-0.61,3.55,2.53,40.44,15.87,0.82,17.85,18.33,1254740000.0,4.229340e+11,0.89,2.45,6.0,23.0,29.0,PETR
CMIN3,5.45,12.10,2.96,1.668,8.16,0.823,3.53,5.46,-3.16,4.66,3.81,30.54,13.78,2.02,26.67,24.47,42052300.0,1.009740e+10,0.91,0.24,18.0,11.0,29.0,CMIN
CSUD3,17.15,7.61,1.41,1.185,16.25,0.960,15.67,5.94,368.94,5.52,3.64,19.95,15.58,1.24,20.52,18.49,1093700.0,5.097310e+08,0.10,4.34,27.0,14.0,41.0,CSUD
CEAB3,12.69,7.40,1.11,0.485,7.12,0.439,2.76,3.85,-3.46,3.92,2.91,12.60,6.56,1.50,16.14,14.95,133664000.0,3.536830e+09,0.35,11.98,10.0,32.0,42.0,CEAB


#### sem as 3 classes

In [105]:
b_print(
    funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
LREN3,13.65,9.87,1.35,0.881,6.46,0.749,3.00,3.94,5.70,3.58,2.61,22.36,8.92,1.76,22.96,13.72,285429000.0,1.014310e+10,0.04,10.59,7.0,13.0,20.0,LREN
PLPL3,13.97,9.01,2.88,0.989,7.20,0.744,1.18,3.88,5.06,3.99,3.88,25.48,13.28,3.70,29.26,31.96,20632200.0,9.895800e+08,1.06,25.95,11.0,9.0,20.0,PLPL
QUAL3,2.23,64.91,0.48,0.429,0.25,0.162,-2.03,1.09,-0.53,2.70,1.74,39.27,1.14,0.81,19.33,0.74,7326760.0,1.325600e+09,1.05,-8.82,3.0,20.0,23.0,QUAL
ASAI3,7.26,10.70,1.72,0.128,1.48,0.218,5.97,1.72,-0.42,3.80,2.91,7.44,1.20,1.12,19.41,16.05,137914000.0,5.720000e+09,2.85,29.34,9.0,19.0,28.0,ASAI
PETR4,30.41,5.06,0.93,0.798,10.63,0.323,-11.88,1.97,-0.61,3.55,2.53,40.44,15.87,0.82,17.85,18.33,1254740000.0,4.229340e+11,0.89,2.45,6.0,23.0,29.0,PETR
CMIN3,5.45,12.10,2.96,1.668,8.16,0.823,3.53,5.46,-3.16,4.66,3.81,30.54,13.78,2.02,26.67,24.47,42052300.0,1.009740e+10,0.91,0.24,18.0,11.0,29.0,CMIN
CSUD3,17.15,7.61,1.41,1.185,16.25,0.960,15.67,5.94,368.94,5.52,3.64,19.95,15.58,1.24,20.52,18.49,1093700.0,5.097310e+08,0.10,4.34,27.0,14.0,41.0,CSUD
CEAB3,12.69,7.40,1.11,0.485,7.12,0.439,2.76,3.85,-3.46,3.92,2.91,12.60,6.56,1.50,16.14,14.95,133664000.0,3.536830e+09,0.35,11.98,10.0,32.0,42.0,CEAB
LEVE3,34.22,7.95,5.29,0.859,7.95,1.177,7.72,5.29,-8.55,6.33,5.54,16.24,10.81,1.31,34.85,66.51,11875700.0,8.769090e+08,1.71,9.91,42.0,6.0,48.0,LEVE
BEEF3,5.72,-7.02,4.43,0.111,2.88,0.126,0.78,1.61,-0.38,4.93,3.91,6.92,-1.57,1.35,17.04,-63.09,100235000.0,1.291830e+09,20.66,14.53,21.0,28.0,49.0,BEEF


outros filtros

In [106]:
funds2 =  funds[funds['P/L'] > 0]                # sem prejuizo atual
funds2 =  funds2[(funds['P/L'] < 30)]           # não estar excessivamente cara
funds2 =  funds2[funds['Dív.Brut/ Patrim.'] < 3.5]  # endividamento sob controle
funds2 =  funds2[funds['Cresc. Rec.5a'] > 0]    # crescimento nos ultimos 5 anos

In [107]:
b_print(
    funds2[(~funds2['Papel'].isin(bancos))&(~funds2['Papel'].isin(seguros))&(~funds2['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
LREN3,13.65,9.87,1.35,0.881,6.46,0.749,3.00,3.94,5.70,3.58,2.61,22.36,8.92,1.76,22.96,13.72,285429000.0,1.014310e+10,0.04,10.59,7.0,13.0,20.0,LREN
PLPL3,13.97,9.01,2.88,0.989,7.20,0.744,1.18,3.88,5.06,3.99,3.88,25.48,13.28,3.70,29.26,31.96,20632200.0,9.895800e+08,1.06,25.95,11.0,9.0,20.0,PLPL
ASAI3,7.26,10.70,1.72,0.128,1.48,0.218,5.97,1.72,-0.42,3.80,2.91,7.44,1.20,1.12,19.41,16.05,137914000.0,5.720000e+09,2.85,29.34,9.0,19.0,28.0,ASAI
PETR4,30.41,5.06,0.93,0.798,10.63,0.323,-11.88,1.97,-0.61,3.55,2.53,40.44,15.87,0.82,17.85,18.33,1254740000.0,4.229340e+11,0.89,2.45,6.0,23.0,29.0,PETR
CMIN3,5.45,12.10,2.96,1.668,8.16,0.823,3.53,5.46,-3.16,4.66,3.81,30.54,13.78,2.02,26.67,24.47,42052300.0,1.009740e+10,0.91,0.24,18.0,11.0,29.0,CMIN
CSUD3,17.15,7.61,1.41,1.185,16.25,0.960,15.67,5.94,368.94,5.52,3.64,19.95,15.58,1.24,20.52,18.49,1093700.0,5.097310e+08,0.10,4.34,27.0,14.0,41.0,CSUD
CEAB3,12.69,7.40,1.11,0.485,7.12,0.439,2.76,3.85,-3.46,3.92,2.91,12.60,6.56,1.50,16.14,14.95,133664000.0,3.536830e+09,0.35,11.98,10.0,32.0,42.0,CEAB
LEVE3,34.22,7.95,5.29,0.859,7.95,1.177,7.72,5.29,-8.55,6.33,5.54,16.24,10.81,1.31,34.85,66.51,11875700.0,8.769090e+08,1.71,9.91,42.0,6.0,48.0,LEVE
POMO3,5.67,5.89,1.64,0.774,20.20,0.725,2.47,5.01,11.25,5.91,5.31,15.45,13.25,1.91,20.09,27.78,7101830.0,4.330200e+09,0.75,28.12,35.0,17.0,52.0,POMO
VTRU3,13.85,3.95,0.66,0.833,0.16,0.304,2.84,2.37,-0.87,4.49,3.50,35.17,21.06,2.34,15.04,16.58,8096450.0,2.834840e+09,0.86,8.81,16.0,38.0,54.0,VTRU


retirar BDR

In [108]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds[funds['Papel'].str.contains(padrao)]


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
48,AURA33,99.9,-85.1,12.65,4.956,0.83,2.855,21.06,10.37,-6.82,...,1.62,41.33,-14.87,48342900.0,1.714890e+09,1.33,21.31,119.0,3.0,122.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.

In [109]:
data_funds.loc['INEP4']

Cotação              1.060000e+00
P/L                 -1.200000e-01
P/VP                -3.000000e-02
PSR                  1.285600e+01
Div.Yield            0.000000e+00
P/Ativo              1.180000e-01
P/Cap.Giro          -5.000000e-02
P/EBIT              -5.600000e-01
P/Ativ Circ.Liq     -2.000000e-02
EV/EBIT             -1.048000e+01
EV/EBITDA           -1.123000e+01
Mrg Ebit            -2.280750e+03
Mrg. Líq.           -1.070710e+04
Liq. Corr.           1.300000e-01
ROIC                -2.648000e+01
ROE                  2.384000e+01
Liq.2meses           2.816480e+04
Patrim. Líq         -1.791040e+09
Dív.Brut/ Patrim.   -5.200000e-01
Cresc. Rec.5a       -2.459000e+01
Name: INEP4, dtype: float64

==================================================================================================

In [110]:
# ============================================================
# MAGIC FORMULA 
# ============================================================

df_mf = df.copy()

# Converter para numérico
cols = ['EV/EBIT', 'ROIC', 'Liq.2meses']
for c in cols:
    df_mf[c] = pd.to_numeric(df_mf[c], errors='coerce')

# Filtros básicos (Brasil)
df_mf = df_mf[
    (df_mf['EV/EBIT'] > 0) &
    (df_mf['ROIC'] > 0) &
    (df_mf['Liq.2meses'] > 1_000_000)
]

# Earnings Yield implícito
df_mf['earnings_yield'] = 1 / df_mf['EV/EBIT']

# Rankings (Greenblatt)
df_mf['rank_ey'] = df_mf['earnings_yield'].rank(ascending=False)
df_mf['rank_roic'] = df_mf['ROIC'].rank(ascending=False)

df_mf['magic_score'] = df_mf['rank_ey'] + df_mf['rank_roic']

df_magic = df_mf.sort_values('magic_score')

carteira = (
    df_magic
    .loc[:, [
        'Papel',
        'EV/EBIT',
        'ROIC',
        'earnings_yield',
        'magic_score',
        'Liq.2meses'
    ]]
)

carteira.head(15)


,Papel,EV/EBIT,ROIC,earnings_yield,magic_score,Liq.2meses
1,PSSA3,0.57,78.18,1.754386,2.0,9.025130e+07
2,WIZC3,2.23,35.46,0.448430,6.0,3.350840e+06
3,LREN3,3.58,22.96,0.279330,14.0,2.854290e+08
4,PLPL3,3.99,29.26,0.250627,15.0,2.063220e+07
8,CMIN3,4.66,26.67,0.214592,21.0,4.205230e+07
6,ASAI3,3.80,19.41,0.263158,22.0,1.379140e+08
7,PETR4,3.55,17.85,0.281690,22.5,1.254740e+09
9,PETR3,3.66,17.85,0.273224,24.5,3.890580e+08
10,CSUD3,5.52,20.52,0.181159,31.5,1.093700e+06
11,CEAB3,3.92,16.14,0.255102,32.5,1.336640e+08


In [111]:
df_score = df.copy()

cols_num = [
    'EV/EBIT', 'P/L', 'P/VP',
    'ROIC', 'Mrg Ebit', 'ROE',
    'Cresc. Rec.5a', 'Liq.2meses'
]

for c in cols_num:
    df_score[c] = pd.to_numeric(df_score[c], errors='coerce')

# Filtros mínimos
df_score = df_score[
    (df_score['EV/EBIT'] > 0) &
    (df_score['P/L'] > 0) &
    (df_score['ROIC'] > 0) &
    (df_score['Liq.2meses'] > 1_000_000)
]

# Value score
df_score['rank_ev_ebit'] = df_score['EV/EBIT'].rank(ascending=True)
df_score['rank_pl'] = df_score['P/L'].rank(ascending=True)
df_score['rank_pvp'] = df_score['P/VP'].rank(ascending=True)

df_score['value_score'] = (
    df_score['rank_ev_ebit'] +
    df_score['rank_pl'] +
    df_score['rank_pvp']
)

# Quality score
df_score['rank_roic'] = df_score['ROIC'].rank(ascending=False)
df_score['rank_mrg'] = df_score['Mrg Ebit'].rank(ascending=False)
df_score['rank_roe'] = df_score['ROE'].rank(ascending=False)

df_score['quality_score'] = (
    df_score['rank_roic'] +
    df_score['rank_mrg'] +
    df_score['rank_roe']
)

# growth score
df_score['rank_growth'] = df_score['Cresc. Rec.5a'].rank(ascending=False)

df_score['growth_score'] = df_score['rank_growth']

# Score final (pesos claros)
# Sugestão equilibrada para position trade:
# Value: 40%
# Quality: 40%
# Growth: 20%

# Normalizar scores (0–1)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_score[['value_n', 'quality_n', 'growth_n']] = scaler.fit_transform(
    df_score[['value_score', 'quality_score', 'growth_score']]
)

# Score final (quanto MENOR, melhor)
df_score['final_score'] = (
    0.4 * df_score['value_n'] +
    0.4 * df_score['quality_n'] +
    0.2 * df_score['growth_n']
)

df_final = df_score.sort_values('final_score')
carteira = (
    df_final
    .loc[:, [
        'Papel',
        'EV/EBIT',
        'P/L',
        'P/VP',
        'ROIC',
        'Mrg Ebit',
        'ROE',
        'Cresc. Rec.5a',
        'final_score',
        'Liq.2meses'
    ]]
)

carteira.head(15)


,Papel,EV/EBIT,P/L,P/VP,ROIC,Mrg Ebit,ROE,Cresc. Rec.5a,final_score,Liq.2meses
2,WIZC3,2.23,7.65,2.08,35.46,50.71,27.23,13.39,0.221562,3.350840e+06
15,VTRU3,4.49,3.95,0.66,15.04,35.17,16.58,8.81,0.226041,8.096450e+06
1,PSSA3,0.57,9.64,2.08,78.18,81.71,21.54,17.08,0.233723,9.025130e+07
14,POMO3,5.91,5.89,1.64,20.09,15.45,27.78,28.12,0.238168,7.101830e+06
7,PETR4,3.55,5.06,0.93,17.85,40.44,18.33,2.45,0.239600,1.254740e+09
4,PLPL3,3.99,9.01,2.88,29.26,25.48,31.96,25.95,0.243685,2.063220e+07
9,PETR3,3.66,5.34,0.98,17.85,40.44,18.33,2.45,0.249533,3.890580e+08
18,POMO4,6.21,6.25,1.74,20.09,15.45,27.78,28.12,0.258698,1.325800e+08
38,COGN3,5.19,4.98,0.52,10.54,32.38,10.43,12.57,0.277529,1.002660e+08
72,MDNE3,6.17,6.22,1.21,8.51,19.63,19.53,33.89,0.284461,2.405620e+07


Interpretação correta
Score menor = melhor

Value garante margem de segurança

Quality reduz value trap

Growth evita empresas “baratas que não crescem”

==================================================================================================